# Run pyCIAM

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from warnings import catch_warnings, filterwarnings

from pyCIAM.run import execute_pyciam
from shared import (
    AUTHOR,
    CONTACT,
    DIR_SCRATCH,
    HISTORY,
    PATH_PARAMS,
    PATH_SLIIDERS,
    PATH_SLIIDERS_SEG,
    PATH_SLR_INT,
    PATHS_SURGE_LOOKUP,
    PATT_OUTPUTS,
    PATT_REFA,
    RES_VERS,
    start_dask_cluster,
)

/srv/conda/envs/notebook/lib/python3.13/site-packages/rhg_compute_tools/__init__.py:6: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [3]:
TMPPATH = DIR_SCRATCH / f"pyCIAM_results_quantiles_prechunked_{RES_VERS}_dr_{{dr}}.zarr"

DR = [0.015, 0.02, 0.03]

# optimized for the Code Ocean machines (8 CPU / 64 GB)
SEG_CHUNKSIZE = 9

SEG_ADM_SUBSET = None

DESCRIPTION = "Projected coastal damages from pyCIAM, using quantiles of SLR scenarios."

In [6]:
client, cluster = start_dask_cluster()
client

Connection method: Cluster object,Cluster type: dask_gateway.GatewayCluster
Dashboard: https://notebooks.rhgresearch.com/services/dask-gateway/clusters/jhub.b1f9061ee4564231a09e36aa822a8cda/status,


In [7]:
with catch_warnings():
    filterwarnings("ignore", module="distributed.worker.memory")
    for dr in DR:
        print(dr)
        execute_pyciam(
            PATH_PARAMS,
            PATH_SLIIDERS,
            [PATH_SLR_INT],
            ["is"],
            PATT_REFA.parent / PATT_REFA.name.format(dr=dr),
            econ_input_path_seg=PATH_SLIIDERS_SEG,
            output_path=PATT_OUTPUTS.parent / PATT_OUTPUTS.name.format(dr=dr)
            if SEG_ADM_SUBSET is None
            else None,
            seg_var_subset=SEG_ADM_SUBSET,
            surge_input_paths=PATHS_SURGE_LOOKUP,
            tmp_output_path=TMPPATH.parent / TMPPATH.name.format(dr=dr),
            pyciam_seg_chunksize=SEG_CHUNKSIZE,
            quantiles=[0.5],
            dask_client_func=lambda: client,
            remove_tmpfile=False,
            params_override={"dr": dr},
            extra_attrs={
                "author": AUTHOR,
                "contact": CONTACT,
                "description": DESCRIPTION,
                "history": HISTORY,
            },
        )

cluster.close(), client.close()

/srv/conda/envs/notebook/lib/python3.13/site-packages/zarr/codecs/vlen_utf8.py:44: UserWarning: The codec `vlen-utf8` is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  return cls(**configuration_parsed)
/srv/conda/envs/notebook/lib/python3.13/site-packages/zarr/core/array.py:4282: UserWarning: The dtype `<U15` is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  meta = AsyncArray._create_metadata_v3(
/srv/conda/envs/notebook/lib/python3.13/site-packages/zarr/core/array.py:4282: UserWarning: The dtype `<U5` is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  meta = AsyncArray._create_metadata_v3(
/srv/conda/envs/notebook/lib/python3.13/site-packages/zarr/codecs/vlen_utf8.py:44: UserWarning: The codec `vlen-utf8` is

(None, None)